In [124]:
import sys

import monai
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim

import pandas as pd
import numpy as np

from tqdm import tqdm

from MINIT.minit import MINiT

import os

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [90]:
adni_merge_1 = pd.read_csv('our_subj_adnimerge.csv').sort_values(['PTID', 'EXAMDATE']).reset_index(drop=True)
adni_merge_2 = pd.read_csv('ADNIMERGE.csv').sort_values(['PTID', 'EXAMDATE']).reset_index(drop=True)
adni_merge_1 = adni_merge_1[['PTID', 'VISCODE', 'DX_bl']]
adni_merge_2 = adni_merge_2[['PTID', 'VISCODE', 'DX_bl']]

/tmp/ipykernel_1301/9726765.py:2: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  adni_merge_2 = pd.read_csv('ADNIMERGE.csv').sort_values(['PTID', 'EXAMDATE']).reset_index(drop=True)


In [91]:
# какое количество классов во всем ADNI
from collections import Counter
Counter(pd.read_csv('ADNIMERGE.csv')['DX_bl'].values)

/tmp/ipykernel_1301/2513726996.py:3: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  Counter(pd.read_csv('ADNIMERGE.csv')['DX_bl'].values)


Counter({'CN': 4512,
         'AD': 1667,
         'LMCI': 5037,
         'SMC': 1037,
         'EMCI': 2740,
         nan: 10})

In [60]:
pd.read_csv('ADNIMERGE.csv')

/tmp/ipykernel_1301/4009778700.py:1: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('ADNIMERGE.csv')


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36665,NaN,NaN,0.000000,0.00000,0,0.0,2020-01-09 04:20:17.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,239.7,22.83,1.08355,NaN,NaN,0.000000,0.00000,0,0.0,2020-01-09 04:20:17.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,239.7,22.83,1.08355,NaN,NaN,0.498289,5.96721,6,6.0,2020-03-25 15:43:58.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,239.7,22.83,1.08355,NaN,NaN,0.999316,11.96720,12,12.0,2020-01-09 04:20:17.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,239.7,22.83,1.08355,NaN,NaN,1.998630,23.93440,24,24.0,2020-01-09 04:20:17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,6309,114_S_6309,m24,114,ADNI3,ADNI3,2020-10-27,SMC,65.2,Female,...,NaN,NaN,NaN,NaN,1.0199,2.379190,28.49180,30,24.0,2020-11-06 04:21:15.0
14999,6385,024_S_6385,m24,24,ADNI3,ADNI3,2020-11-05,CN,66.6,Female,...,NaN,NaN,NaN,NaN,NaN,2.414780,28.91800,30,24.0,2020-11-06 04:21:15.0
15000,6574,941_S_6574,m24,941,ADNI3,ADNI3,2020-10-27,CN,73.5,Female,...,NaN,NaN,NaN,NaN,NaN,2.108140,25.24590,24,24.0,2020-11-06 04:21:15.0
15001,6575,941_S_6575,m24,941,ADNI3,ADNI3,2020-10-27,CN,73.5,Male,...,NaN,NaN,NaN,NaN,NaN,2.108140,25.24590,24,24.0,2020-11-06 04:21:15.0


In [14]:
# какое количество классов в нашем сэмпле ADNI
Counter(pd.read_csv('our_subj_adnimerge.csv')['DX_bl'].values)

Counter({'CN': 775, 'AD': 409, 'LMCI': 1430})

In [72]:
Counter(pd.read_csv('our_subj_adnimerge.csv')['COLPROT'].values)

Counter({'ADNI1': 2613, 'ADNI2': 1})

# Делаем dataset

In [223]:
BASE_PATH = '/home/druzhininapo/nfs/caps/subjects/'

In [235]:
def make_dataset(adni_merge, resize=64):
    resizer_ = monai.transforms.Resize((resize, resize, resize))
    label2num = {
        'AD': 1,
        'LMCI': 0,
    }
    dataset = {}
    for subj in tqdm(np.unique(adni_merge_1[['PTID']].values)):
        subj_path = subj.replace('_', '')
        for ses in os.listdir(os.path.join(BASE_PATH, f"sub-ADNI{subj_path}")):
            subj_samples = adni_merge[adni_merge['PTID'] == subj]
            if subj_samples[subj_samples['VISCODE'] == ses.split('-')[1].lower()].shape[0] == 0:
                continue
            label_text = subj_samples[subj_samples['VISCODE'] == ses.split('-')[1].lower()]['DX_bl'].values[0]
            if label_text not in label2num:
                continue
            
            path_to_tensor = os.path.join(BASE_PATH, f"sub-ADNI{subj_path}", ses, 'deeplearning_prepare_data/image_based/t1_linear/')
            image_tensor = torch.load(os.path.join(path_to_tensor, os.listdir(path_to_tensor)[0]))
            
            if resize:
                image_tensor = resizer_(image_tensor).get_array()

            if subj in dataset:
                dataset[subj].append((image_tensor, label2num[label_text]))
            else:
                dataset[subj] = [(image_tensor, label2num[label_text])]
    
    return dataset

In [236]:
dataset_dict = make_dataset(adni_merge_1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 771/771 [07:23<00:00,  1.74it/s]


# Train

In [237]:
class MRIDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.labels = labels
        self.images = images

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image = torch.tensor(self.images[index])

        return {'tensor': image, 'label': self.labels[index]}

In [238]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, roc_auc_score

def get_dataloder(dataset_dict, config_params):

    def make_tensors(set_subj):
        X = []
        y = []
        for subj in set_subj:
            for tensor, label in dataset_dict[subj]:
                X.append(tensor)
                y.append(label)

        return X, y
    
    all_keys_subj = np.sort(list(dataset_dict.keys()))
    kfold = KFold(n_splits=config_params['training_params']['count_folds_splits'], random_state=29, shuffle=True)
    for i, (train_idx, test_idx) in enumerate(kfold.split(all_keys_subj)):
        
        if config_params['training_params']['verbose_train']:
            print(f"Number fold: {i+1}")
    
        X_train, y_train = make_tensors(all_keys_subj[train_idx])
        X_test, y_test = make_tensors(all_keys_subj[test_idx])

        mri_dataset_train = MRIDataset(X_train, y_train)
        mri_dataset_test = MRIDataset(X_test, y_test)

        mri_dataloader_train = torch.utils.data.DataLoader(mri_dataset_train,
                                                           batch_size=config_params['training_params']['batch_size'],
                                                           shuffle=True)
        mri_dataloader_test = torch.utils.data.DataLoader(mri_dataset_test, batch_size=1, shuffle=False)
    
        yield mri_dataloader_train, mri_dataloader_test

In [239]:
def train_vit(dataset_dict, config_params, thr=0.5, use_checkpoint=False):
    auc_folds = []

    if not os.path.exists(config_params['result_path']):
        os.mkdir(config_params['result_path'])

    file_log = open(os.path.join(config_params['result_path'], 'logs.txt'), 'w')

    print(f"Name of exp: {config_params['name_exp']}")
    file_log.write(f"Name of exp: {config_params['name_exp']}\n")

    device = torch.device(config_params['global_params']['device'] if torch.cuda.is_available() else "cpu")

    max_auc_folds = []
    num_fold = 0
    for train_dataloader, test_dataloader in get_dataloder(dataset_dict,
                                                           config_params,):
        num_fold += 1
        file_log.write(f"Fold: {num_fold}\n")

        if config_params['model_params']['model_name'] == 'MiNiT':
            model = MINiT(
                block_size = config_params['model_params']['block_size'],
                image_size = config_params['model_params']['image_size'],
                patch_size = config_params['model_params']['patch_size'],
                num_classes = config_params['model_params']['num_classes'],
                channels = config_params['model_params']['channels'],
                dim = config_params['model_params']['dim'],
                depth = config_params['model_params']['depth'],
                heads = config_params['model_params']['heads'],
                mlp_dim = config_params['model_params']['mlp_dim'],
            ).train().to(device)
        elif config_params['model_params']['model_name'] == 'ViT':
            model = OwnViT(
                in_channels = config_params['model_params']['in_channels'],
                img_size = (config_params['model_params']['img_size'],
                            config_params['model_params']['img_size'],
                            config_params['model_params']['img_size']),
                patch_size = (config_params['model_params']['patch_size'],
                              config_params['model_params']['patch_size'],
                              config_params['model_params']['patch_size']),
                num_layers = config_params['model_params']['num_layers'],
                num_heads = config_params['model_params']['num_heads'],
                hidden_size = config_params['model_params']['hidden_size'],
                pos_embed = config_params['model_params']['pos_embed'],
                qkv_bias = config_params['model_params']['qkv_bias'],
                classification = config_params['model_params']['classification'],
                num_classes = config_params['model_params']['num_classes'],
                post_activation = config_params['model_params']['post_activation'],
                spatial_dims = config_params['model_params']['spatial_dims'],
            ).train().to(device)

        if config_params['training_params']['use_checkpoint']:
            checkpoint = torch.load("MINIT/minit.pt")
            state = checkpoint['model_state_dict']

        optimizer = optim.Adam(model.parameters(), lr=config_params['training_params']['lr'])
        loss = nn.BCELoss()

        auc_folds = []
        max_auc = -1.0

        for epoch in range(config_params['training_params']['epoch_num']):
            print(f"Train step")
            file_log.write(f"Train step\n")

            model.train()
            epoch_train_loss = 0.0
            for i, batch in enumerate(train_dataloader):
                brain_img = batch['tensor'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()

                if config_params['model_params']['model_name'] == 'MiNiT':
                    outputs = model(brain_img)
                elif config_params['model_params']['model_name'] == 'ViT':
                    outputs, _ = model(brain_img)
                loss_calc = loss(torch.squeeze(outputs, 1), labels.float())

                loss_calc.backward()
                optimizer.step()

                epoch_train_loss += loss_calc.item()
                if i % 24 == 0:
                    print(f"Epoch: {epoch+1}, step: {i+1} / {len(train_dataloader)}, loss: {epoch_train_loss / (i+1)}")
                    file_log.write(f"Epoch: {epoch+1}, step: {i+1} / {len(train_dataloader)}, loss: {epoch_train_loss / (i+1)}\n")

            print(f"Test step")
            file_log.write(f"Test step\n")

            model.eval()
            #labels_pred = []
            logits_pred = []
            y_test = []
            for i, batch in enumerate(test_dataloader):
                brain_img = batch['tensor'].to(device)
                labels = batch['label'].to(device)

                with torch.no_grad():
                    if config_params['model_params']['model_name'] == 'MiNiT':
                        output_prob = torch.sigmoid(model(brain_img)).item()
                    elif config_params['model_params']['model_name'] == 'ViT':
                        output_prob = model(brain_img)[0].item()
                    logits_pred.append(output_prob)
                    y_test.append(labels.item())
                    #print(output_prob, labels.item())
                    #labels_pred.append(1 if output_prob > config_params['training_params']['thr_metirc'] else 0)
      
            #precision_score_fold = precision_score(y_test, labels_pred)
            #recall_score_fold = recall_score(y_test, labels_pred)
            auc_fold = roc_auc_score(y_test, logits_pred)

            if auc_fold > max_auc:
                max_auc = auc_fold
                torch.save(model.state_dict(),
                           os.path.join(config_params['result_path'],
                           f"best_model_fold_{num_fold}.pth"))

            auc_folds.append(auc_fold)

            print(f"AUC: {auc_fold}")
            file_log.write(f"AUC: {auc_fold}\n")

        max_auc_folds.append(max_auc)

    file_log.write(f"Mean AUC: {np.mean(max_auc_folds)}")
    file_log.close()
    return auc_folds, max_auc_folds

In [ ]:
vit_model = ViT(in_channels=1,
   img_size=64,
   patch_size=8,
               classification=True,
               num_classes=1,
               )

In [ ]:
name_config = 'ad_cd_vit_3.yaml'
#name_config = 'ad_lmci_minit.yaml'

In [240]:
name_config = 'ad_lmci_minit.yaml'

In [241]:
import yaml

with open(os.path.join("./exp_configs", name_config), "r") as stream:
    try:
        config_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [242]:
config_params

{'name_exp': 'AD vs LMCI MiNiT',
 'result_path': './results/ad_lmci_minit/',
 'image_size': 64,
 'global_params': {'device': 'cuda:0'},
 'model_params': {'model_name': 'MiNiT',
  'block_size': 8,
  'image_size': 64,
  'patch_size': 8,
  'num_classes': 1,
  'channels': 1,
  'dim': 256,
  'depth': 6,
  'heads': 4,
  'mlp_dim': 309},
 'training_params': {'lr': 0.005,
  'count_folds_splits': 10,
  'use_checkpoint': False,
  'verbose_make_dataset': True,
  'verbose_train': True,
  'thr_metirc': 0.5,
  'batch_size': 6,
  'epoch_num': 4}}

In [243]:
auc_folds, max_auc_folds = train_vit(dataset_dict, config_params)

Name of exp: AD vs LMCI MiNiT
Number fold: 1
Train step
Epoch: 1, step: 1 / 232, loss: 1.3578466176986694
Epoch: 1, step: 25 / 232, loss: 2.6825997638702392
Epoch: 1, step: 49 / 232, loss: 2.614070858274187
Epoch: 1, step: 73 / 232, loss: 2.3800291427194256
Epoch: 1, step: 97 / 232, loss: 2.2295218890475246
Epoch: 1, step: 121 / 232, loss: 2.2001498356338374
Epoch: 1, step: 145 / 232, loss: 2.088273521127372
Epoch: 1, step: 169 / 232, loss: 2.077956741378152
Epoch: 1, step: 193 / 232, loss: 2.1526555014397815
Epoch: 1, step: 217 / 232, loss: 2.1163320458979102
Test step
Precision: 0.23255813953488372
Recall: 1.0
AUC: 0.5263888888888889
Train step
Epoch: 2, step: 1 / 232, loss: 2.0327186584472656
Epoch: 2, step: 25 / 232, loss: 2.009307804107666
Epoch: 2, step: 49 / 232, loss: 1.8960718086787633
Epoch: 2, step: 73 / 232, loss: 1.8858051712382329
Epoch: 2, step: 97 / 232, loss: 1.9898126816626676
Epoch: 2, step: 121 / 232, loss: 2.0293867097905847
Epoch: 2, step: 145 / 232, loss: 1.98426

/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.0
Recall: 0.0
AUC: 0.6319865319865321
Train step
Epoch: 4, step: 1 / 232, loss: 2.2407066822052
Epoch: 4, step: 25 / 232, loss: 1.8719015854597092
Epoch: 4, step: 49 / 232, loss: 1.6237434598864342
Epoch: 4, step: 73 / 232, loss: 1.7573372074594236
Epoch: 4, step: 97 / 232, loss: 1.630548845246895
Epoch: 4, step: 121 / 232, loss: 1.684220245061827
Epoch: 4, step: 145 / 232, loss: 1.637322079801354
Epoch: 4, step: 169 / 232, loss: 1.6312375528704839
Epoch: 4, step: 193 / 232, loss: 1.6818576925892594
Epoch: 4, step: 217 / 232, loss: 1.6402529451807248
Test step


/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.0
Recall: 0.0
AUC: 0.7079966329966331
Number fold: 2
Train step
Epoch: 1, step: 1 / 232, loss: 1.3888145685195923
Epoch: 1, step: 25 / 232, loss: 2.1595096683502195
Epoch: 1, step: 49 / 232, loss: 2.093582802889298
Epoch: 1, step: 73 / 232, loss: 2.0394390609166395
Epoch: 1, step: 97 / 232, loss: 2.1495864520367887
Epoch: 1, step: 121 / 232, loss: 2.0968118733610988
Epoch: 1, step: 145 / 232, loss: 2.0246567027322175
Epoch: 1, step: 169 / 232, loss: 2.0630850097131446
Epoch: 1, step: 193 / 232, loss: 2.0889182879053867
Epoch: 1, step: 217 / 232, loss: 2.052993082025084
Test step
Precision: 0.23255813953488372
Recall: 1.0
AUC: 0.5
Train step
Epoch: 2, step: 1 / 232, loss: 0.0
Epoch: 2, step: 25 / 232, loss: 2.098764533996582
Epoch: 2, step: 49 / 232, loss: 1.9539299838396968
Epoch: 2, step: 73 / 232, loss: 1.8403961462517306
Epoch: 2, step: 97 / 232, loss: 1.7636509903312958
Epoch: 2, step: 121 / 232, loss: 1.6888412761786753
Epoch: 2, step: 145 / 232, loss: 1.6923681656701

...

Precision: 0.0
Recall: 0.0
AUC: 0.7664141414141413
Train step
Epoch: 2, step: 1 / 232, loss: 2.781757354736328
Epoch: 2, step: 25 / 232, loss: 2.85910626411438
Epoch: 2, step: 49 / 232, loss: 2.4436018856204287
Epoch: 2, step: 73 / 232, loss: 2.3778408047271102
Epoch: 2, step: 97 / 232, loss: 2.2640933744686165
Epoch: 2, step: 121 / 232, loss: 2.240818966518749
Epoch: 2, step: 145 / 232, loss: 2.2104716342071007
Epoch: 2, step: 169 / 232, loss: 2.1411801516657043
Epoch: 2, step: 193 / 232, loss: 2.055691071124892
Epoch: 2, step: 217 / 232, loss: 1.9912152627852107
Test step
Precision: 0.6666666666666666
Recall: 0.03333333333333333
AUC: 0.7784511784511785
Train step
Epoch: 3, step: 1 / 232, loss: 4.036620140075684
Epoch: 3, step: 25 / 232, loss: 1.9904972076416017
Epoch: 3, step: 49 / 232, loss: 1.8598138312904202
Epoch: 3, step: 73 / 232, loss: 1.9110685782889798
Epoch: 3, step: 97 / 232, loss: 1.75399086678151
Epoch: 3, step: 121 / 232, loss: 1.6864744945744838
Epoch: 3, step: 145 / 2

/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.0
Recall: 0.0
AUC: 0.7724747474747475
Train step
Epoch: 4, step: 1 / 232, loss: 0.0
Epoch: 4, step: 25 / 232, loss: 1.3823825538158416
Epoch: 4, step: 49 / 232, loss: 1.49912413224882
Epoch: 4, step: 73 / 232, loss: 1.3697787008873403
Epoch: 4, step: 97 / 232, loss: 1.423785741488958
Epoch: 4, step: 121 / 232, loss: 1.6208926482079935
Epoch: 4, step: 145 / 232, loss: 1.6808821093162587
Epoch: 4, step: 169 / 232, loss: 1.6402667978414769
Epoch: 4, step: 193 / 232, loss: 1.749392500716633
Epoch: 4, step: 217 / 232, loss: 1.7195528170720498
Test step
Precision: 0.23255813953488372
Recall: 1.0
AUC: 0.723526936026936
Number fold: 8
Train step
Epoch: 1, step: 1 / 232, loss: 1.3678829669952393
Epoch: 1, step: 25 / 232, loss: 1.9638892269134522
Epoch: 1, step: 49 / 232, loss: 1.950443956316734
Epoch: 1, step: 73 / 232, loss: 2.037104750741018
Epoch: 1, step: 97 / 232, loss: 2.0901171684879616
Epoch: 1, step: 121 / 232, loss: 2.0803780045883715
Epoch: 1, step: 145 / 232, loss: 2.05

/root/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.0
Recall: 0.0
AUC: 0.5844276094276094
Train step
Epoch: 4, step: 1 / 232, loss: 3.7439463138580322
Epoch: 4, step: 25 / 232, loss: 2.362349452972412
Epoch: 4, step: 49 / 232, loss: 2.0911717937917125
Epoch: 4, step: 73 / 232, loss: 2.077650920168994
Epoch: 4, step: 97 / 232, loss: 2.100173246307471
Epoch: 4, step: 121 / 232, loss: 2.138475878425866
Epoch: 4, step: 145 / 232, loss: 2.160326509023535
Epoch: 4, step: 169 / 232, loss: 2.1025703695751505
Epoch: 4, step: 193 / 232, loss: 2.050212743214375
Epoch: 4, step: 217 / 232, loss: 2.036305677918245
Test step
Precision: 1.0
Recall: 0.06666666666666667
AUC: 0.7673400673400673


In [244]:
np.mean(max_auc_folds)

0.715820707070707

# AutoEnc ViT

In [ ]:
from typing import Sequence, Union

from monai.networks.nets import ViT, ViTAutoEnc
from monai.networks.layers import Conv
from monai.utils import ensure_tuple_rep


class MVitAutoEnc(nn.Module):
    def __init__(self, vit,
                in_channels: int,
                img_size: Union[Sequence[int], int],
                patch_size: Union[Sequence[int], int],
                out_channels: int = 1,
                deconv_chns: int = 16,
                hidden_size: int = 768,
                mlp_dim: int = 3072,
                num_layers: int = 12,
                num_heads: int = 12,
                pos_embed: str = "conv",
                dropout_rate: float = 0.0,
                spatial_dims: int = 3,):
        super().__init__()

        self.vit = vit

        self.patch_size = ensure_tuple_rep(patch_size, spatial_dims)
        self.spatial_dims = spatial_dims
        
        new_patch_size = [4] * self.spatial_dims
        conv_trans = Conv[Conv.CONVTRANS, self.spatial_dims]
        # self.conv3d_transpose* is to be compatible with existing 3d model weights.
        self.conv3d_transpose = conv_trans(hidden_size,
                                           deconv_chns,
                                           kernel_size=new_patch_size,
                                           stride=new_patch_size)
        self.conv3d_transpose_1 = conv_trans(
            in_channels=deconv_chns, out_channels=out_channels, kernel_size=new_patch_size, stride=new_patch_size
        )
        
    def forward(self, x):
        spatial_size = x.shape[2:]
        _, hidden_states = self.vit(x)

        x = self.vit.norm(hidden_states[-1])
        
        x = x.transpose(1, 2)
        d = [s // p for s, p in zip(spatial_size, self.patch_size)]
        x = torch.reshape(x, [x.shape[0], x.shape[1], *d])
        x = self.conv3d_transpose(x)
        x = self.conv3d_transpose_1(x)
        
        return x


class MVitCls(nn.Module):
    def __init__(self, vit,
                 hidden_size: int = 768,
                 num_classes: int = 2,):
        super().__init__()

        self.vit = vit
        self.cls_token = nn.Parameter(torch.zeros(1, 1, hidden_size))
        self.classification_head = nn.Sequential(nn.Linear(hidden_size, num_classes), nn.Tanh())

    def forward(self, x):
        x, _ = self.vit(x)
        x = self.classification_head(x[:, 0])
        return x

In [ ]:
from monai.networks.nets import ViT

vit_model = ViT(in_channels=1,
                img_size=(64, 64, 64),
                patch_size=(16, 16, 16),
                num_layers=4,
                pos_embed='conv',
                qkv_bias=False,
                classification=False,
                spatial_dims=3,)


mvit = MVitAutoEnc(vit=vit_model,
            in_channels=1,
            img_size=(64, 64, 64),
            patch_size=(16, 16, 16),
            spatial_dims=3,)


mvit_cls = MVitCls(vit=mvit.vit,
                   hidden_size=768,
                   num_classes=1,)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, roc_auc_score

def get_dataloder(dataset_dict, batch_train_size=4, n_splits=5, verbose=True):

    def make_tensors(set_subj):
        X = []
        y = []
        for subj in set_subj:
            for tensor, label in dataset_dict[subj]:
                X.append(tensor)
                y.append(label)

        return X, y
    
    all_keys_subj = np.sort(list(dataset_dict.keys()))
    kfold = KFold(n_splits=n_splits, random_state=29, shuffle=True)
    for i, (train_idx, test_idx) in enumerate(kfold.split(all_keys_subj)):
        
        if verbose:
            print(f"Number fold: {i+1}")
    
        X_train, y_train = make_tensors(all_keys_subj[train_idx])
        X_test, y_test = make_tensors(all_keys_subj[test_idx])

        mri_dataset_train = MRIDataset(X_train, y_train)
        mri_dataset_test = MRIDataset(X_test, y_test)

        mri_dataloader_train = torch.utils.data.DataLoader(mri_dataset_train, batch_size=batch_train_size, shuffle=True)
        mri_dataloader_test = torch.utils.data.DataLoader(mri_dataset_test, batch_size=1, shuffle=False)
    
        yield mri_dataloader_train, mri_dataloader_test

In [ ]:
mvit.to(device)

In [ ]:
def train_auto_enc_vit(dataset_dict, config_params, thr=0.5, use_checkpoint=False):
    auc_folds = []

    if not os.path.exists(config_params['result_path']):
        os.mkdir(config_params['result_path'])

    file_log = open(os.path.join(config_params['result_path'], 'logs.txt'), 'w')

    print(f"Name of exp: {config_params['name_exp']}")
    file_log.write(f"Name of exp: {config_params['name_exp']}\n")

    device = torch.device(config_params['global_params']['device'] if torch.cuda.is_available() else "cpu")

    max_auc_folds = []
    num_fold = 0
    for train_dataloader, test_dataloader in get_dataloder(dataset_dict,
                                                           config_params,):
        num_fold += 1
        file_log.write(f"Fold: {num_fold}\n")

        if config_params['model_params']['model_name'] == 'ViT':
            model = OwnViT(
                in_channels = config_params['model_params']['in_channels'],
                img_size = (config_params['model_params']['img_size'],
                            config_params['model_params']['img_size'],
                            config_params['model_params']['img_size']),
                patch_size = (config_params['model_params']['patch_size'],
                              config_params['model_params']['patch_size'],
                              config_params['model_params']['patch_size']),
                num_layers = config_params['model_params']['num_layers'],
                num_heads = config_params['model_params']['num_heads'],
                hidden_size = config_params['model_params']['hidden_size'],
                pos_embed = config_params['model_params']['pos_embed'],
                qkv_bias = config_params['model_params']['qkv_bias'],
                classification = config_params['model_params']['classification'],
                num_classes = config_params['model_params']['num_classes'],
                post_activation = config_params['model_params']['post_activation'],
                spatial_dims = config_params['model_params']['spatial_dims'],
            ).train().to(device)

        if config_params['training_params']['use_checkpoint']:
            checkpoint = torch.load("MINIT/minit.pt")
            state = checkpoint['model_state_dict']

        optimizer = optim.Adam(model.parameters(), lr=config_params['training_params']['lr'])
        loss = nn.BCELoss()

        auc_folds = []
        max_auc = -1.0

        for epoch in range(config_params['training_params']['epoch_num']):
            print(f"Train step")
            file_log.write(f"Train step\n")

            model.train()
            epoch_train_loss = 0.0
            for i, batch in enumerate(train_dataloader):
                brain_img = batch['tensor'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()

                if config_params['model_params']['model_name'] == 'MiNiT':
                    outputs = model(brain_img)
                elif config_params['model_params']['model_name'] == 'ViT':
                    outputs, _ = model(brain_img)
                loss_calc = loss(torch.squeeze(outputs, 1), labels.float())

                loss_calc.backward()
                optimizer.step()

                epoch_train_loss += loss_calc.item()
                if i % 24 == 0:
                    print(f"Epoch: {epoch+1}, step: {i+1} / {len(train_dataloader)}, loss: {epoch_train_loss / (i+1)}")
                    file_log.write(f"Epoch: {epoch+1}, step: {i+1} / {len(train_dataloader)}, loss: {epoch_train_loss / (i+1)}\n")

            print(f"Test step")
            file_log.write(f"Test step\n")

            model.eval()
            #labels_pred = []
            logits_pred = []
            y_test = []
            for i, batch in enumerate(test_dataloader):
                brain_img = batch['tensor'].to(device)
                labels = batch['label'].to(device)

                with torch.no_grad():
                    if config_params['model_params']['model_name'] == 'MiNiT':
                        output_prob = torch.sigmoid(model(brain_img)).item()
                    elif config_params['model_params']['model_name'] == 'ViT':
                        output_prob = model(brain_img)[0].item()
                    logits_pred.append(output_prob)
                    y_test.append(labels.item())

            auc_fold = roc_auc_score(y_test, logits_pred)

            if auc_fold > max_auc:
                max_auc = auc_fold
                torch.save(model.state_dict(),
                           os.path.join(config_params['result_path'],
                           f"best_model_fold_{num_fold}.pth"))

            auc_folds.append(auc_fold)

            print(f"AUC: {auc_fold}")
            file_log.write(f"AUC: {auc_fold}\n")

        max_auc_folds.append(max_auc)

    file_log.write(f"Mean AUC: {np.mean(max_auc_folds)}")
    file_log.close()
    return auc_folds, max_auc_folds

# OLD

In [243]:
# non-pretrain + finetune
from sklearn.metrics import precision_score, recall_score

for epoch in range(3):
    net.train()
    epoch_train_loss = 0.0
    for i, batch in enumerate(mri_dataloader_train):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = net(brain_img)
        loss_calc = loss(torch.squeeze(outputs), labels.float())

        loss_calc.backward()
        optimizer.step()
        
        epoch_train_loss += loss_calc.item()
        if i % 8 == 0:
            print(f"Epoch: {epoch}, step: {i+1} / {len(mri_dataloader_train)}, loss: {epoch_train_loss / (i+1)}")

    net.eval()
    labels_pred = []
    for i, batch in enumerate(mri_dataloader_test):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)

        with torch.no_grad():
            output_prob = torch.sigmoid(net(brain_img)).item()
            if output_prob > 0.5:
                labels_pred.append(1)
            else:
                labels_pred.append(0)

    print(precision_score(y_test, labels_pred))
    print(recall_score(y_test, labels_pred))

Epoch: 0, step: 1 / 281, loss: 0.0
Epoch: 0, step: 9 / 281, loss: 2.394197834862603
Epoch: 0, step: 17 / 281, loss: 2.000979598830728
Epoch: 0, step: 25 / 281, loss: 2.072249717712402
Epoch: 0, step: 33 / 281, loss: 1.838512633786057
Epoch: 0, step: 41 / 281, loss: 1.8390611381065556
Epoch: 0, step: 49 / 281, loss: 1.676484385315253
Epoch: 0, step: 57 / 281, loss: 1.7525349311661302
Epoch: 0, step: 65 / 281, loss: 1.7216973625696623
Epoch: 0, step: 73 / 281, loss: 1.7372005336905179
Epoch: 0, step: 81 / 281, loss: 1.7519290660634452
Epoch: 0, step: 89 / 281, loss: 1.7796537909614907
Epoch: 0, step: 97 / 281, loss: 1.7823393240417402
Epoch: 0, step: 105 / 281, loss: 1.7946016629536947
Epoch: 0, step: 113 / 281, loss: 1.804586623622253
Epoch: 0, step: 121 / 281, loss: 1.7872062537295759
Epoch: 0, step: 129 / 281, loss: 1.7722801626190658
Epoch: 0, step: 137 / 281, loss: 1.7688390703967019
Epoch: 0, step: 145 / 281, loss: 1.7726228578337309
Epoch: 0, step: 153 / 281, loss: 1.7720017296816

/opt/miniconda-latest/envs/neuro/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0
0.0
Epoch: 2, step: 1 / 281, loss: 0.9903404712677002
Epoch: 2, step: 9 / 281, loss: 1.2091296911239624
Epoch: 2, step: 17 / 281, loss: 1.2393223608241362
Epoch: 2, step: 25 / 281, loss: 1.4782209968566895
Epoch: 2, step: 33 / 281, loss: 1.306702633247231
Epoch: 2, step: 41 / 281, loss: 1.8242759279361584
Epoch: 2, step: 49 / 281, loss: 1.6870434719080827
Epoch: 2, step: 57 / 281, loss: 1.7964918945442165
Epoch: 2, step: 65 / 281, loss: 1.912631487617126
Epoch: 2, step: 73 / 281, loss: 1.8508511142779702
Epoch: 2, step: 81 / 281, loss: 1.8840244005859634
Epoch: 2, step: 89 / 281, loss: 1.8810518161299523
Epoch: 2, step: 97 / 281, loss: 1.924387781736777
Epoch: 2, step: 105 / 281, loss: 1.8459877461194991
Epoch: 2, step: 113 / 281, loss: 1.8139593773974783
Epoch: 2, step: 121 / 281, loss: 1.7698975948016507
Epoch: 2, step: 129 / 281, loss: 1.7190615355737449
Epoch: 2, step: 137 / 281, loss: 1.7104708726932532
Epoch: 2, step: 145 / 281, loss: 1.6974194407976906
Epoch: 2, step: 153 / 

In [168]:
# non-pretrain + finetune
from sklearn.metrics import precision_score, recall_score

for epoch in range(3):
    net.train()
    epoch_train_loss = 0.0
    for i, batch in enumerate(mri_dataloader_train):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = net(brain_img)
        loss_calc = loss(torch.squeeze(outputs), labels.float())

        loss_calc.backward()
        optimizer.step()
        
        epoch_train_loss += loss_calc.item()
        if i % 8 == 0:
            print(f"Epoch: {epoch}, step: {i+1} / {len(mri_dataloader_train)}, loss: {epoch_train_loss / (i+1)}")

    net.eval()
    labels_pred = []
    for i, batch in enumerate(mri_dataloader_test):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)

        with torch.no_grad():
            output_prob = torch.sigmoid(net(brain_img)).item()
            if output_prob > 0.5:
                labels_pred.append(1)
            else:
                labels_pred.append(0)

    print(precision_score(y_test, labels_pred))
    print(recall_score(y_test, labels_pred))

Epoch: 0, step: 1 / 281, loss: 2.7592720985412598
Epoch: 0, step: 9 / 281, loss: 2.0186812612745495
Epoch: 0, step: 17 / 281, loss: 2.0448199720943676
Epoch: 0, step: 25 / 281, loss: 1.946508674621582
Epoch: 0, step: 33 / 281, loss: 1.8931593822710442
Epoch: 0, step: 41 / 281, loss: 1.893749219615285
Epoch: 0, step: 49 / 281, loss: 1.892821985848096
Epoch: 0, step: 57 / 281, loss: 1.9896142169048912
Epoch: 0, step: 65 / 281, loss: 1.8508722672095665
Epoch: 0, step: 73 / 281, loss: 1.9114093437586746
Epoch: 0, step: 81 / 281, loss: 1.9593559150342588
Epoch: 0, step: 89 / 281, loss: 1.9997915217045987
Epoch: 0, step: 97 / 281, loss: 1.9489044049351485
Epoch: 0, step: 105 / 281, loss: 1.9180229425430297
Epoch: 0, step: 113 / 281, loss: 1.9272019588841802
Epoch: 0, step: 121 / 281, loss: 1.9498930420757326
Epoch: 0, step: 129 / 281, loss: 1.9260694749595584
Epoch: 0, step: 137 / 281, loss: 1.9041858955021322
Epoch: 0, step: 145 / 281, loss: 1.9132682200135855
Epoch: 0, step: 153 / 281, los

/opt/miniconda-latest/envs/neuro/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0
0.0
Epoch: 1, step: 1 / 281, loss: 2.830392599105835
Epoch: 1, step: 9 / 281, loss: 1.83727731804053
Epoch: 1, step: 17 / 281, loss: 1.517961924566942
Epoch: 1, step: 25 / 281, loss: 1.7089318430423737
Epoch: 1, step: 33 / 281, loss: 1.7376287480195363
Epoch: 1, step: 41 / 281, loss: 1.6466180776677481
Epoch: 1, step: 49 / 281, loss: 1.5208125582763128
Epoch: 1, step: 57 / 281, loss: 1.5473935023734444
Epoch: 1, step: 65 / 281, loss: 1.5701666919084696
Epoch: 1, step: 73 / 281, loss: 1.5076581325433025
Epoch: 1, step: 81 / 281, loss: 1.4787089151364785
Epoch: 1, step: 89 / 281, loss: 1.4306895120090313
Epoch: 1, step: 97 / 281, loss: 1.4951415943730737
Epoch: 1, step: 105 / 281, loss: 1.4685016708714622
Epoch: 1, step: 113 / 281, loss: 1.4720258272327154
Epoch: 1, step: 121 / 281, loss: 1.4305070916975826
Epoch: 1, step: 129 / 281, loss: 1.4437457232974296
Epoch: 1, step: 137 / 281, loss: 1.465366628483264
Epoch: 1, step: 145 / 281, loss: 1.5115189178236599
Epoch: 1, step: 153 / 28

In [159]:
# pretrain + finetune
from sklearn.metrics import precision_score, recall_score

for epoch in range(2):
    net.train()
    epoch_train_loss = 0.0
    for i, batch in enumerate(mri_dataloader_train):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = net(brain_img)
        loss_calc = loss(torch.squeeze(outputs), labels.float())

        loss_calc.backward()
        optimizer.step()
        
        epoch_train_loss += loss_calc.item()
        if i % 8 == 0:
            print(f"Epoch: {epoch}, step: {i+1} / {len(mri_dataloader_train)}, loss: {epoch_train_loss / (i+1)}")

    net.eval()
    labels_pred = []
    for i, batch in enumerate(mri_dataloader_test):
        brain_img = batch['tensor'].to(device)
        labels = batch['label'].to(device)

        with torch.no_grad():
            output_prob = torch.sigmoid(net(brain_img)).item()
            if output_prob > 0.5:
                labels_pred.append(1)
            else:
                labels_pred.append(0)

    print(precision_score(y_test, labels_pred))
    print(recall_score(y_test, labels_pred))

Epoch: 0, step: 1 / 281, loss: 1.7101575136184692
Epoch: 0, step: 5 / 281, loss: 1.4812070608139039
Epoch: 0, step: 9 / 281, loss: 1.3079716364542644
Epoch: 0, step: 13 / 281, loss: 1.823077495281513
Epoch: 0, step: 17 / 281, loss: 1.6630467527052935
Epoch: 0, step: 21 / 281, loss: 1.8416676975431896
Epoch: 0, step: 25 / 281, loss: 1.9523288106918335
Epoch: 0, step: 29 / 281, loss: 2.014886194262011
Epoch: 0, step: 33 / 281, loss: 1.8870762586593628
Epoch: 0, step: 37 / 281, loss: 1.8546467729516931
Epoch: 0, step: 41 / 281, loss: 1.9063204148920572
Epoch: 0, step: 45 / 281, loss: 1.9453119436899822
Epoch: 0, step: 49 / 281, loss: 2.035807441691963
Epoch: 0, step: 53 / 281, loss: 1.9134045974263605
Epoch: 0, step: 57 / 281, loss: 1.9582718485280086
Epoch: 0, step: 61 / 281, loss: 1.9239012882357738
Epoch: 0, step: 65 / 281, loss: 1.876353223507221
Epoch: 0, step: 69 / 281, loss: 1.8896263388619907
Epoch: 0, step: 73 / 281, loss: 1.8255659619422808
Epoch: 0, step: 77 / 281, loss: 1.8008

In [94]:
torch.cuda.is_available()

True